# Has Many Movie Lab

### Introduction
In this lab we will continue to look at the "Has-Many" relationships in our data. The database we will be using during this lab contains information about a selection of movies and related entities such as actors, directors and writers. A movie entity will have relationships with actor, director, and writer entities. The actors, directors and writers will also have relationships with themselves (i.e. a director will have worked with many actors). In problems below, we will use our knowledge of these relationships to build SQL queries.

Let's begin by connecting to the database and reviewing the schema of the tables.

In [1]:
import sqlite3
conn = sqlite3.connect('movie_films_actors.db')
cursor = conn.cursor()

In [2]:
import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-movies-lab/"
names = ['actors', 'directors', 'movies', 'writers', 'movie_actors', 'movie_directors', 'movie_writers']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]

In [3]:
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False)

In [4]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('actors',),
 ('directors',),
 ('movies',),
 ('writers',),
 ('movie_actors',),
 ('movie_directors',),
 ('movie_writers',)]

In [5]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'studio', 'TEXT', 0, None, 0),
 (3, 'runtime', 'REAL', 0, None, 0),
 (4, 'description', 'TEXT', 0, None, 0),
 (5, 'release_date', 'TEXT', 0, None, 0),
 (6, 'year', 'INTEGER', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(actors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(directors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(writers)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(movie_actors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'actor_id', 'INTEGER', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(movie_directors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'director_id', 'INTEGER', 0, None, 0)]

In [ ]:
cursor.execute('PRAGMA table_info(movie_writers)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'writer_id', 'INTEGER', 0, None, 0)]

Let's start off with some basic one table queries:

* What is the title, length, and id of the movie with the longest runtime?

In [ ]:
cursor.execute("select movies.id, movies.title, movies.runtime from movies where runtime = (select max(movies.runtime) from movies)")
cursor.fetchall()
# [('Never Sleep Again: The Elm Street Legacy', 480.0, 11415)]

[(11415, 'Never Sleep Again: The Elm Street Legacy', 480.0)]

* Using your answer from the previous question, how many actors were credited for the movie with the longest runtime? Hint: Use the COUNT function with the movie ID

In [ ]:
cursor.execute("select movies.id, movies.title, movies.runtime, count(actor_id) from movies join movie_actors on movie_actors.movie_id = movies.id where runtime = (select max(movies.runtime) from movies) ")
cursor.fetchall()

# [(6,)]

[(11415, 'Never Sleep Again: The Elm Street Legacy', 480.0, 6)]

* What was the shortest movie released in 2006?

In [ ]:
cursor.execute("select movies.title, movies.runtime from movies order by movies.runtime asc limit 1")


#cursor.execute("select movies.title, movies.runtime from movies where movies.title = 'The Guardian'")
cursor.fetchall()
# [('The Guardian',)]

[('12 Strong', None)]

### Has Many

* What are the names of the actors in Toy Story?

In [ ]:
cursor.execute("select movies.title,actors.name from movies join movie_actors on movie_actors.movie_id = movies.id join actors on actors.id = movie_actors.actor_id where movies.title ='Toy Story'")
cursor.fetchall()
# [('Tom Hanks',),
#  ('Jim Varney',),
#  ('Wallace Shawn',),
#  ('Don Rickles',),
#  ('John Ratzenberger',),
#  ('Tim Allen',)]

[('Toy Story', 'Tom Hanks'),
 ('Toy Story', 'Tim Allen'),
 ('Toy Story', 'Jim Varney'),
 ('Toy Story', 'Wallace Shawn'),
 ('Toy Story', 'Don Rickles'),
 ('Toy Story', 'John Ratzenberger')]

* What is the name of the director of Toy Story?

In [ ]:
#cursor.execute("""select * from movies 
 #  join  movie_directors on movie_directors.movie_id = movies.id 
 # join directors on directors.id = movie_directors.movie_id 
 # where movies.title = 'Toy Story' """)

cursor.execute("""select * from movie_directors 
  join movies on movies.id = movie_directors.movie_id
  join directors on directors.id = movie_directors.director_id
  where movies.title = 'Toy Story'""")
cursor.fetchall()

# [('John Lasseter',)]

[(28668,
  3648,
  141,
  3648,
  'Toy Story',
  'Buena Vista',
  80.0,
  'Woody, a traditional pull-string talking cowboy, has long enjoyed a place of honor as the favorite among six-year-old Andy\'s menagerie of toys. Quick to calm their anxieties about being replaced by newer arrivals, Woody finds his own confidence shaken, and his status as top toy in jeopardy, upon the arrival of Buzz Lightyear, simply the coolest space action figure ever made. Woody plots to get rid of Buzz, but things backfire and he finds himself lost in the outside world with Buzz as his only companion. Joining forces to find their way home, the two rivals set out on an adventure that lands them in the clutches of Sid, a sadistic neighborhood kid who is notorious for dismembering and reassembling "mutant" toys in his bedroom. As "guests" of Sid and his dog, Scud, the two fugitive toys forge a genuine friendship and learn that only through mutual trust and respect do they have any chance of survival.',
  '1995-

In [ ]:
[(3648,
  'Toy Story',
  'Buena Vista',
  80.0,
  'Woody, a traditional pull-string talking cowboy, has long enjoyed a place of honor as the favorite among six-year-old Andy\'s menagerie of toys. Quick to calm their anxieties about being replaced by newer arrivals, Woody finds his own confidence shaken, and his status as top toy in jeopardy, upon the arrival of Buzz Lightyear, simply the coolest space action figure ever made. Woody plots to get rid of Buzz, but things backfire and he finds himself lost in the outside world with Buzz as his only companion. Joining forces to find their way home, the two rivals set out on an adventure that lands them in the clutches of Sid, a sadistic neighborhood kid who is notorious for dismembering and reassembling "mutant" toys in his bedroom. As "guests" of Sid and his dog, Scud, the two fugitive toys forge a genuine friendship and learn that only through mutual trust and respect do they have any chance of survival.',
  '1995-11-22 00:00:00',
  1995,
  28668,
  3648,
  141,
  3648,
  'Penelope Spheeris')]

[(3648,
  'Toy Story',
  'Buena Vista',
  80.0,
  'Woody, a traditional pull-string talking cowboy, has long enjoyed a place of honor as the favorite among six-year-old Andy\'s menagerie of toys. Quick to calm their anxieties about being replaced by newer arrivals, Woody finds his own confidence shaken, and his status as top toy in jeopardy, upon the arrival of Buzz Lightyear, simply the coolest space action figure ever made. Woody plots to get rid of Buzz, but things backfire and he finds himself lost in the outside world with Buzz as his only companion. Joining forces to find their way home, the two rivals set out on an adventure that lands them in the clutches of Sid, a sadistic neighborhood kid who is notorious for dismembering and reassembling "mutant" toys in his bedroom. As "guests" of Sid and his dog, Scud, the two fugitive toys forge a genuine friendship and learn that only through mutual trust and respect do they have any chance of survival.',
  '1995-11-22 00:00:00',
  1995,

* What are the names of the writers of Toy Story?

In [ ]:
cursor.execute("""select writers.name from writers join movie_writers on movie_writers.writer_id = writers.id 
join movies on movies.id = movie_writers.movie_id
where movies.title = 'Toy Story'""")
cursor.fetchall()


# [('Joss Whedon',), ('Joel Cohen',), ('Andrew Stanton',), ('Alec Sokolow',)]

[('Joss Whedon',), ('Joel Cohen',), ('Andrew Stanton',), ('Alec Sokolow',)]

* What is the name and actor id of the actor with the most credits in the database?

In [ ]:
cursor.execute("""
select actors.name, actors.id, count(actors.name) as bob from actors 
join movie_actors on movie_actors.actor_id = actors.id 
join movies on movies.id = movie_actors.movie_id
group by actors.name
order by bob desc
limit 1
""")
cursor.fetchall()

# [('Robert De Niro', 429, 78)]

[('Robert De Niro', 429, 78)]

* What are the titles of the movies the actor from the previous question has been in, after the year 2005?

In [ ]:
cursor.execute("""
select movies.title, movies.release_date from movies
join movie_actors on movie_actors.movie_id = movies.id
join actors on actors.id = movie_actors.actor_id
where actors.name = "Robert De Niro"
and movies.release_date >= 2006
order by movies.release_date asc
""")
cursor.fetchall()


# [("New Year's Eve",),
#  ('Mr. Warmth: The Don Rickles Project',),
#  ('Hands of Stone',),
#  ('Last Vegas',),
#  ('I Knew It Was You: Rediscovering John Cazale',),
#  ('Stardust',),
#  ('Killer Elite',),
#  ("Everybody's Fine",),
#  ('Stone',),
#  ('Machete',),
#  ('Red Lights',),
#  ('Righteous Kill',),
#  ('The Good Shepherd',),
#  ('The Bag Man',),
#  ('Being Flynn',),
#  ('Joy',),
#  ('The Wizard of Lies',),
#  ('Limitless',),
#  ('Killing Season',),
#  ('The Family',),
#  ('Heist',),
#  ('Great Expectations',),
#  ('Little Fockers',),
#  ('What Just Happened?',),
#  ('The Comedian',),
#  ('The Big Wedding',),
#  ('Dirty Grandpa',),
#  ('Grudge Match',)]

[('The Good Shepherd', '2006-12-22 00:00:00'),
 ('Stardust', '2007-08-10 00:00:00'),
 ('What Just Happened?', '2008-01-19 00:00:00'),
 ('Mr. Warmth: The Don Rickles Project', '2008-02-19 00:00:00'),
 ('Righteous Kill', '2008-09-10 00:00:00'),
 ('I Knew It Was You: Rediscovering John Cazale', '2009-01-16 00:00:00'),
 ("Everybody's Fine", '2009-12-04 00:00:00'),
 ('Machete', '2010-09-03 00:00:00'),
 ('Stone', '2010-10-08 00:00:00'),
 ('Little Fockers', '2010-12-22 00:00:00'),
 ('Limitless', '2011-03-18 00:00:00'),
 ('Killer Elite', '2011-09-23 00:00:00'),
 ("New Year's Eve", '2011-12-09 00:00:00'),
 ('Being Flynn', '2012-03-02 00:00:00'),
 ('Red Lights', '2012-07-13 00:00:00'),
 ('The Big Wedding', '2013-04-26 00:00:00'),
 ('Killing Season', '2013-07-12 00:00:00'),
 ('The Family', '2013-09-13 00:00:00'),
 ('Last Vegas', '2013-11-01 00:00:00'),
 ('Great Expectations', '2013-11-08 00:00:00'),
 ('Grudge Match', '2013-12-25 00:00:00'),
 ('The Bag Man', '2014-02-28 00:00:00'),
 ('Heist', '201

* What are the titles of movies with more than two directors -- order by title ascending and limit to the first five results

In [ ]:
cursor.execute("""select movies.title, directors.name, count(movies.title) from movies
  join movie_directors on movies.id = movie_directors.movie_id
  join directors on directors.id = movie_directors.director_id
  group by movies.title
  having count(movies.title) >1
  order by movies.title asc
  limit 5
 """)
cursor.fetchall()


# [('101 Dalmatians',),
#  ('11/8/2016',),
#  ('A Crude Awakening: The Oil Crash',),
#  ('A Farewell To Arms',),
#  ("A Liar's Autobiography - The Untrue Story of Monty Python's Graham Chapman",)]

[('...So Goes the Nation', 'James D. Stern', 2),
 ('100 Bloody Acres', 'Colin Cairnes', 2),
 ('101 Dalmatians', 'Stephen Herek', 4),
 ("101 Dalmatians II: Patch's London Adventure", 'Jim Kammerud', 2),
 ('11/8/2016', 'Yung Chang', 3)]

### Has Many Through

* What is the name of the writer in the database that has been credited the most times during the year 2018?

In [ ]:
cursor.execute(""" select writers.name, count(writers.id) from writers
join movie_writers on movie_writers.writer_id = writers.id
join movies on movies.id = movie_writers.movie_id
where movies.year = 2018

group by writers.id

order by count(writers.id) desc
limit 1
""")
cursor.fetchall()


# [('Ryan Engle', 3)]

[('Ryan Engle', 3)]

* What is the name of the actor or actress in the database that has been credited the most between 2010 and 2015 (inclusive)?

In [ ]:
cursor.execute(""" select actors.name, count(actors.id) from actors
join movie_actors on movie_actors.actor_id = actors.id 
join movies on movies.id = movie_actors.movie_id 
where movies.year >=2010 and movies.year <=2015
group by actors.id
order by count(actors.id) desc
limit 1

""")
cursor.fetchall()
# [('James Franco', 22)]

[('James Franco', 22)]

* What are the names of all actors who performed in more than 3 movies in 2010?

In [ ]:
cursor.execute(""" select actors.name, count(actors.id) from actors
join movie_actors on movie_actors.actor_id = actors.id 
join movies on movies.id = movie_actors.movie_id 
where movies.year = 2010
group by actors.id
having count(actors.id) >3


""")
cursor.fetchall()

# [('Aaron Taylor-Johnson',),
#  ('Adam Scott',),
#  ('Barry Pepper',),
#  ('Ben Stiller',),
#  ('Danny Huston',),
#  ('Gemma Arterton',),
#  ('Helen Mirren',),
#  ('Jay Baruchel',),
#  ('Jessica Alba',),
#  ('Jonah Hill',),
#  ('Josh Brolin',),
#  ('Josh Duhamel',),
#  ('Keith David',),
#  ('Liam Neeson',),
#  ('Matt Damon',),
#  ('Melissa Leo',),
#  ('Patricia Clarkson',),
#  ('Pierce Brosnan',),
#  ('Ralph Fiennes',),
#  ('Susan Sarandon',),
#  ('Zach Galifianakis',)]

[('Liam Neeson', 7),
 ('Ben Stiller', 4),
 ('Pierce Brosnan', 5),
 ('Patricia Clarkson', 4),
 ('Barry Pepper', 4),
 ('Matt Damon', 4),
 ('Josh Brolin', 5),
 ('Ralph Fiennes', 5),
 ('Jonah Hill', 4),
 ('Jay Baruchel', 4),
 ('Helen Mirren', 4),
 ('Danny Huston', 4),
 ('Gemma Arterton', 4),
 ('Aaron Taylor-Johnson', 4),
 ('Susan Sarandon', 5),
 ('Josh Duhamel', 4),
 ('Jessica Alba', 4),
 ('Adam Scott', 5),
 ('Zach Galifianakis', 5),
 ('Melissa Leo', 5),
 ('Keith David', 4)]

* What studio has Steven Spielberg worked with the most?

In [ ]:
cursor.execute(""" select studio, count(studio) from movies

join movie_directors on movies.id = movie_directors.movie_id 
join directors on movie_directors.director_id = directors.id 

where directors.name = "Steven Spielberg"
group by studio
order by count(studio) desc
limit 1

""")
cursor.fetchall()

# [('Universal Pictures', 7)]

[('Universal Pictures', 7)]

* What years did Steven Spielberg direct 2 movies?

In [ ]:
cursor.execute(""" select movies.year, count(movies.year) from movies

join movie_directors on movies.id = movie_directors.movie_id 
join directors on movie_directors.director_id = directors.id 

where directors.name = "Steven Spielberg"
group by movies.year
having count(movies.year) = 2
""")
cursor.fetchall()
# [(1989, 2), (1993, 2), (1997, 2), (2002, 2), 
# (2005, 2), (2011, 2), (2018, 2)]

[(1989, 2), (1993, 2), (1997, 2), (2002, 2), (2005, 2), (2011, 2), (2018, 2)]

* How many movies has each of the actors from Toy Story been in? (movie ID is 3648)

In [ ]:
cursor.execute(""" select movies.title, actors.name, count(actors.name) from actors 
join movie_actors on movie_actors.actor_id = actors.id
join movies on movies.id = movie_actors.movie_id

where actors.name in (select actors.name from actors 
join movie_actors on movie_actors.actor_id = actors.id
join movies on movies.id = movie_actors.movie_id
where movies.title ="Toy Story")

group by actors.name
order by count(actors.name)desc
""")
# cursor.execute("""select actors.name from actors 
# join movie_actors on movie_actors.actor_id = actors.id
# join movies on movies.id = movie_actors.movie_id
# where movies.title ="Toy Story" """)
cursor.fetchall()

# [('Tom Hanks', 46),
 # ('Jim Varney', 8),
 # ('Wallace Shawn', 27),
 # ('Don Rickles', 10),
 # ('John Ratzenberger', 7),
 # ('Tim Allen', 20)]

[('The Da Vinci Code', 'Tom Hanks', 46),
 ('The Princess Bride', 'Wallace Shawn', 27),
 ('The Santa Clause', 'Tim Allen', 20),
 ("Kelly's Heroes", 'Don Rickles', 10),
 ('3 Ninjas: High Noon at Mega Mountain', 'Jim Varney', 8),
 ('Up', 'John Ratzenberger', 7)]

* What are the names of other movies the director of Toy Story directed?

In [ ]:
cursor.execute(""" select movies.title, directors.name from movies
join movie_directors on movie_directors.movie_id = movies.id
join directors on directors.id = movie_directors.director_id

where directors.name = (select directors.name from directors 
join movie_directors on movie_directors.director_id = directors.id
join movies on movies.id = movie_directors.movie_id
where movies.title ="Toy Story")

""")
cursor.fetchall()

# [('Cars 2',), ('Cars',), ("A Bug's Life",), ('Toy Story 2',), ('Toy Story',)]

[('Cars 2', 'John Lasseter'),
 ('Cars', 'John Lasseter'),
 ("A Bug's Life", 'John Lasseter'),
 ('Toy Story 2', 'John Lasseter'),
 ('Toy Story', 'John Lasseter')]

* What are the names of all the directors Tom Hanks has worked with? (Actor ID 189) -- order by the director's name ascending and limit to the first five results.

In [ ]:
cursor.execute(""" select directors.name from directors
join movie_directors on movie_directors.director_id = directors.id
join movies on movies.id = movie_directors.movie_id

join movie_actors on movie_actors.movie_id = movies.id 

where movie_actors.actor_id = 189
order by directors.name asc 
limit 5
""")
cursor.fetchall()

# [('Alexander Mackendrick',),
#  ('Angus MacLane',),
#  ('Brian DePalma',),
#  ('Chris Paine',),
#  ('Clint Eastwood',)]

[('Alexander Mackendrick',),
 ('Angus MacLane',),
 ('Brian DePalma',),
 ('Chris Paine',),
 ('Clint Eastwood',)]

* What is the name of the director Tom Hanks has worked with the most?

In [ ]:
cursor.execute(""" select directors.name, count(directors.id) from directors
join movie_directors on movie_directors.director_id = directors.id
join movies on movies.id = movie_directors.movie_id

join movie_actors on movie_actors.movie_id = movies.id 

where movie_actors.actor_id = 189
group by directors.id
order by count(directors.id) desc
limit 1
""")
cursor.fetchall()

# [('Steven Spielberg', 5)]

[('Steven Spielberg', 5)]

* What are the names of all the writers Tom Hanks has worked with?

In [ ]:
cursor.execute(""" select writers.name from writers
join movie_writers on movie_writers.writer_id = writers.id 
join movies on movies.id = movie_writers.movie_id

join movie_actors on movie_actors.movie_id = movies.id 

where movie_actors.actor_id = 189
group by writers.id

""")
cursor.fetchall()

# [('Eric Roth',),
#  ('Nia Vardalos',),
#  ('Tom Hanks',),
#  ('Gary Ross',),
#  ('Anne Spielberg',),
#  ('Chris Paine',),
#  ('Scott Frank',),
#  ('Robert Rodat',),
#  ('Frank Darabont',),
#  ('Tom Tykwer',)]

[('Eric Roth',),
 ('Scott Frank',),
 ('David Koepp',),
 ('Joss Whedon',),
 ('Michael Blodgett',),
 ('John Lasseter',),
 ('Robert Zemeckis',),
 ('Akiva Goldsman',),
 ('Todd Komarnicki',),
 ('Lilly Wachowski',),
 ('Jeff Nathanson',),
 ('Robert Rodat',),
 ('Tom Mankiewicz',),
 ('Al Reinert',),
 ('Dan Aykroyd',),
 ('Lana Wachowski',),
 ('William Broyles',),
 ('John Patrick Shanley',),
 ('Joel Coen',),
 ('Ethan Coen',),
 ('Gary Ross',),
 ('Joel Cohen',),
 ('Dave Eggers',),
 ('Dennis Shryack',),
 ('David Seltzer',),
 ('Steve Purcell (II)',),
 ('Billy Ray',),
 ('Daniel Petrie Jr.',),
 ('Tom Tykwer',),
 ('Andrew Stanton',),
 ('Lee Unkrich',),
 ('Dario Argento',),
 ('Sacha Gervasi',),
 ('William Rose',),
 ('David S. Ward',),
 ('Alan Zweibel',),
 ('Lowell Ganz',),
 ('Nora Ephron',),
 ('Nia Vardalos',),
 ('Robert Klane',),
 ('Bruce Jay Friedman',),
 ('James Ponsoldt',),
 ('Timothy Harris',),
 ('Aaron Sorkin',),
 ('Kevin Pollak',),
 ('Dana Olsen',),
 ('Michael Preminger',),
 ('Kelly Marcel',),
 ('

### Conclusion
The movie database we queried during this lab had a multitude of relationships between the tables. We saw how we could use JOIN to connect the tables, in order query information about entities in different tables. 